## Candidate row creation
The notebook creates all potential rows in the optimal matrix for any $n \in \mathbb{N}$. Depening on available memory, $n$ should be kept below $5$.

In [1]:
import torch
from itertools import combinations, islice, product
import numpy as np

In [4]:
# Generate binary vectors
def vectors(n, binary=False):
    if binary:
        return torch.tensor(list(product([0, 1], repeat=n)), dtype=torch.float32).T
    return torch.tensor(list(product([-1, 1], repeat=n)), dtype=torch.float32).T

# Currently I have rows that appear twice up to a sign change, so this funciton deletes those.
def fix_row_signs(tensor):
    # Iterate over each row
    normalized_tensor = []
    for row in tensor:
        # Find the first non-zero element for sign determination
        first_non_zero_index = (row != 0).nonzero(as_tuple=True)[0][0]
        sign = torch.sign(row[first_non_zero_index])
        normalized_row = row * sign
        normalized_tensor.append(normalized_row)
    normalized_tensor = torch.stack(normalized_tensor)
    # Now return the unique ones.
    return torch.unique(normalized_tensor, dim=0)

# Generates all possible rows appearing in the optimal matrix. Now it's a matter of checking all row combinations.
def candidate_rows(n):
  # Each row represents a subset
  binary_choices = vectors(2**n, binary=True).T

  # Sums over chosen subset to yield all candidate, unnormalized rows
  rows = torch.matmul(binary_choices, vectors(n).T)

  print(f"Start: {rows.shape[0]} rows")

  # Compute their norms
  row_norms = torch.norm(rows, dim=1, keepdim=True)

  # Normalize rows
  normalized_rows_with_nans = rows/row_norms

  # During normalization, some choices add up to 0, so after normalizing we obtain all nan values
  # We thus slice the tensor to remove nan rows
  normalized_rows = normalized_rows_with_nans[~torch.all(normalized_rows_with_nans.isnan(), dim = 1)]

  print(f"Remove zero rows: {normalized_rows.shape[0]} rows left")

  # Remove redundant rows (no need to try them twice!)
  unique_normalized_rows = torch.unique(normalized_rows, dim=0)

  print(f"Remove redundant rows: {unique_normalized_rows.shape[0]} rows left")
  # We can also remove rows that are equal up to a row-wise sign change.
  final_rows = fix_row_signs(unique_normalized_rows)

  print(f"Remove rows up to sign: {final_rows.shape[0]} final rows")
  return final_rows

In [5]:
rows = candidate_rows(4)

Start: 65536 rows
Remove zero rows: 64888 rows left
Remove redundant rows: 1360 rows left
Remove rows up to sign: 680 final rows
